In [1]:
import gc
from src.features_generation import feature_engineering

X_train, y_train, X_test, sample_submission = feature_engineering()

gc.collect()

Timer unit: 1e-06 s

Total time: 82.8116 s
File: /workdir/src/features_generation.py
Function: feature_engineering at line 221

Line #      Hits         Time  Per Hit   % Time  Line Contents
   221                                           @profile
   222                                           def feature_engineering():
   223                                               """
   224                                               load original datasets and conduct feature engineering
   225                                               :return X_train, y_train, X_test, submission:
   226                                               """
   227         1   20343330.0 20343330.0     24.6      train_identity, train_transaction, test_identity, test_transaction, submission = load_data(DATA_DIRECTORY)
   228                                           
   229         1    6425546.0 6425546.0      7.8      train_identity = id_split(train_identity)
   230         1    6602225.0 6602225.0      8

0

In [11]:
X_train.item_id

AttributeError: 'DataFrame' object has no attribute 'item_id'

In [10]:
embed_dim = 128
import torch
from torch_geometric.nn import TopKPooling, SAGEConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
 
        self.conv1 = SAGEConv(embed_dim, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.item_embedding = torch.nn.Embedding(num_embeddings=df.item_id.max() +1, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.item_embedding(x)
        x = x.squeeze(1)        
 
        x = F.relu(self.conv1(x, edge_index))
 
        x, edge_index, _, batch, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
 
        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
 
        x = F.relu(self.conv3(x, edge_index))
 
        x, edge_index, _, batch, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
 
        x = x1 + x2 + x3
 
        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)
 
        x = torch.sigmoid(self.lin3(x)).squeeze(1)
 
        return x

In [8]:
def train():
    model.train()
 
    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)
    
device = torch.device('cuda')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.BCELoss()
train_loader = DataLoader(train_dataset, batch_size=batch_size)
for epoch in range(num_epochs):
    train()

NameError: name 'df' is not defined

In [ ]:
def evaluate(loader):
    model.eval()
 
    predictions = []
    labels = []
 
    with torch.no_grad():
        for data in loader:
 
            data = data.to(device)
            pred = model(data).detach().cpu().numpy()
 
            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

In [ ]:
for epoch in range(100):
    loss = train()
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)    
    test_acc = evaluate(test_loader)
    print('Epoch: {:03d}, Loss: {:.5f}, Train Auc: {:.5f}, Val Auc: {:.5f}, Test Auc: {:.5f}'.
          format(epoch, loss, train_acc, val_acc, test_acc))